In [24]:
import pandas as pd
import json
import re
from fractions import Fraction
import typing
from googletrans import Translator, constants
from datasets import load_dataset
## LOADS AND CACHES DATASET WILL RUN FASTER BEYOND FIRST CALL
dataset = load_dataset("recipe_dataset", split="train")
dataset

Found cached dataset csv (/home/adil/.cache/huggingface/datasets/csv/recipe_dataset-64f57f729428df26/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)


Dataset({
    features: ['Unnamed: 0', 'title', 'ingredients', 'directions', 'link', 'source', 'NER'],
    num_rows: 2231142
})

In [19]:
recipe_df: pd.DataFrame = pd.DataFrame(dataset[:1000])

In [20]:

unitConversions: dict = {
    'c.': '236.588 ml',
    'tsp.': '4.92892 ml',
    'tbsp.': '14.7868 ml',
    'oz.': '29.5735 ml',
    'pt.': '473.176 ml',
    'qt.':' 946.353 ml',
    'lb.': '453.592 grams',
    'gal.': '3785.41 ml',
    ' oz': ' 29.5735 ml',
}

def replace_units(text: str) -> str:
    # replaces american units with european units
    text= text.lower()
    for key, value in unitConversions.items():
        text = text.replace(key, value)
    return text
def replace_fractions(text:str)-> str:
    # replaces fractions with decimals
    text = text.lower()
    text = re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, text)
    return text
def frac2string(s):
    # helper function for replace_fractions
    i, f = s.groups(0)
    f = Fraction(f)
    return str(int(i) + float(f))

# Function to simplify ingredients by only keeping the last word 
def keep_last_word(item):
    words = [x.split()[-1] for x in eval(item)]
    return str(words).replace("'", '"')

def convert_and_multiply_units(text: str) -> list:
    # converts units to floats within strings and multiplies them
    # e.g 1 0.5 0.24 ml becomes 0.12 ml
    ingredients: list = json.loads(text)
    ingredient_list = []
    for ingredient in ingredients: # ingredient is a string with units and one specific ingredient
        ingredient_split: list[str] = re.split(r'\s+|\)|\(', ingredient)
        filtered_result = [item for item in ingredient_split if item]
        converted_values: list= []
        for index, item in enumerate(filtered_result):
            try: # try to turn it into a float and multiply
                converted_value = float(item)
                if len(converted_values) >= 1 and isinstance(converted_values[-1], float):
                    converted_value = converted_values[-1] * converted_value # multiply latest float with current
                    converted_values.pop(-1) # remove latest value, since were using it to multiply
                    converted_values.append(float(converted_value)) 
                else:
                    converted_values.append(converted_value)   
            except ValueError:
                converted_values.append(item)
                pass
            except  IndexError:
                pass
        ingredient_string = " ".join(str(item) for item in converted_values) # join the list back together    
        ingredient_list.append(ingredient_string)
    return ingredient_list

recipe_df['ingredients'] = recipe_df['ingredients'].apply(lambda text: replace_units(text))
recipe_df['ingredients'] = recipe_df['ingredients'].apply(lambda text: replace_fractions(text))
recipe_df['ingredients'] = recipe_df['ingredients'].apply(lambda text: convert_and_multiply_units(text))
recipe_df['NER_simple'] = recipe_df['NER'].apply(keep_last_word)

recipe_df['NER_simple'].head()

0    ["sugar", "milk", "vanilla", "nuts", "butter",...
1                 ["beef", "breasts", "soup", "cream"]
2    ["corn", "cheese", "butter", "powder", "salt",...
3               ["chicken", "gravy", "soup", "cheese"]
4     ["butter", "crumbs", "butter", "sugar", "chips"]
Name: NER_simple, dtype: object

In [26]:
supermarket_df = pd.read_json('supermarket_data/supermarket.json', encoding='UTF-8')
ah_products = pd.DataFrame(supermarket_df.iloc[0]['d'])

# Translate all items from the supermarket data
def translator(output_path='supermarket-data/ah_products_en.csv', write=True) -> None:
  translator = Translator()
  tarray = []

  for product in ah_products['n']:
    translations = translator.translate(product, src="nl", dest="en")
    tarray.append(translations.text)

  ah_products['l_en'] = tarray
  if write:
    ah_products.to_csv(output_path)
    print(f'Wrote translated products to {output_path}')
  else:
    print("Done translating inplace")

def merge_ingredients(df: pd.DataFrame, supermarket_df: pd.DataFrame) -> pd.DataFrame:
  butter_candidates = ah_prod_en[ah_prod_en['l_en'].str.contains('butter')]
  butter_candidates['string_length'] = butter_candidates['l_en'].str.len()

  # Find the length of the shortest string
  min_length = butter_candidates['string_length'].min()

  # Create a boolean mask that is True for rows where the string length is equal to the shortest length
  mask = (butter_candidates['string_length'] == min_length)

  # Apply the mask to get the row(s) with the shortest string in 'l_en'
  shortest_butter_candidates = butter_candidates[mask]
  shortest_butter_candidates


KeyboardInterrupt: 

In [ ]:
recipe_df.head()

In [ ]:

##### DOES NOTHING RIGHT NOW #####

import networkx as nx
from pyvis.network import Network
from IPython.core.display import HTML

# Enable the pyvis interactive display in the Jupyter notebook
from pyvis import network as net
net.template = "notebook"
net.NOTEBOOK = True


# Create a NetworkX graph
G = nx.Graph()
node_list = []
edge_list = []
for key, value in dictTopic.items():
    if data[int(key)].find('country') != None:
        country = data[int(key)].find('country').text
        country = country.replace(f'</country>', "")

        topic = dictTopic[key].replace(f'<topic type="str">', "")
        topic = topic.replace(f'</topic>', "")
        node_list.append(country)
        node_list.append(topic)
        edge_list.append((country, topic))
    else:
        #print("we didn't find a country for this specific article: ", data[int(key)].attrib['id'])
        pass
    
G.add_nodes_from(node_list)
G.add_edges_from(edge_list)

# Create a pyvis network object
net = Network(height="500px", width="100%", bgcolor="#222222", font_color="white", notebook=True)

# Add nodes and edges to the pyvis network object, and set tooltips for each node
for node in G.nodes():
    tooltip = "{}".format(node)
    net.add_node(node, title=tooltip)
for edge in G.edges():
    net.add_edge(*edge, title="hasTopic")
print("Please zoom-in using your scrollwheel and hover over the nodes and edges")
# Display the interactive network graph with tooltips in the Jupyter notebook

net.show("example.html")